<a href="https://colab.research.google.com/github/sergope/tfm/blob/main/prophet_functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install utils

In [ ]:
# import utils

In [ ]:
# Conectar el notebook con la cuenta de gdrive
from google.colab import drive
drive.mount('/content/drive/', force_remount=False)

BASE_FOLDER = 'drive/My Drive/TFM/resources/'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
from fbprophet.diagnostics import cross_validation
from fbprophet.diagnostics import performance_metrics
from fbprophet.plot import plot_cross_validation_metric


In [ ]:
%run 'drive/My Drive/TFM/normalize_data_functions'.ipynb
%run 'drive/My Drive/TFM/aemet'.ipynb

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# def prophet_prediction(df, periods_number, options):
#   m = Prophet()
#   if options and options["holidays"]:
#     m.add_country_holidays(country_name='Spain')

#   m.fit(df)
#   future = m.make_future_dataframe(periods=periods_number)
#   future.tail()

#   forecast = m.predict(future)
#   print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail())

#   fig1 = m.plot(forecast)
#   fig2 = m.plot_components(forecast)
#   # fig3 = plot_plotly(m, forecast)

#   return forecast

In [ ]:
def prophet_prediction(df, periods_number, options):
  yearly_order = options["yearly_order"] if options and options["yearly_order"] else 10
  weekly_order = options["weekly_order"] if options and options["weekly_order"] else 10
  m = Prophet(weekly_seasonality=weekly_order, yearly_seasonality = yearly_order)
  
  if options and options["holidays"]:
    m.add_country_holidays(country_name='Spain')


  df['on_workday'] = df['ds'].apply(is_workday)
  df['off_workday'] = ~df['ds'].apply(is_workday)
  m.add_seasonality(name='weekly_on_workday', period=7, fourier_order=10, condition_name='on_workday')
  m.add_seasonality(name='weekly_off_workday', period=7, fourier_order=3, condition_name='off_workday')

  m.fit(df)
  future = m.make_future_dataframe(periods=periods_number)
  future.tail()

  future['on_workday'] = future['ds'].apply(is_workday)
  future['off_workday'] = ~future['ds'].apply(is_workday)

  forecast = m.predict(future)
  print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail())

  if options and options["cross_validations"]:
    df_cv = cross_validation(m, initial='730 days', period='180 days', horizon = '365 days', parallel="processes")
    # df_cv = cross_validation(m, horizon = '365 days', parallel="processes")
    df_p = performance_metrics(df_cv)
    print(df_p)
    if options["figs_cross_validations"]:
      for fig in options["figs_cross_validations"]:
        plot_cross_validation_metric(df_cv, metric = fig)

  if options and options["plot_figures"]:
    fig1 = m.plot(forecast)
    fig2 = m.plot_components(forecast)
    # fig3 = plot_plotly(m, forecast)

  return forecast

In [ ]:
def predict_one_magnitude(file_name, magnitude, periods, options):
  df = prophet_data_normalized_one_magnitude(file_name, magnitude)
  forecast = prophet_prediction(df, periods, options)

  return {'forecast': forecast, 'df': df}

In [ ]:
def is_workday(ds):
    date = pd.to_datetime(ds)
    return (date.dayofweek > 4)


In [ ]:
def prophet_prediction_with_regresors(df, periods_number, regressors, future_aemet, options):
  yearly_order = options["yearly_order"] if options and options["yearly_order"] else 10
  weekly_order = options["weekly_order"] if options and options["weekly_order"] else 10
  # m = Prophet(weekly_seasonality=weekly_order, yearly_seasonality = yearly_order)
  m = Prophet(weekly_seasonality=False, yearly_seasonality = yearly_order)

  if options and options["holidays"]:
    m.add_country_holidays(country_name='Spain')

  for regressor in regressors:
    # m.add_regressor(regressor, prior_scale=0.5, mode='multiplicative')
    # m.add_regressor(regressor, prior_scale=2)
    m.add_regressor(regressor)


  df['on_workday'] = df['ds'].apply(is_workday)
  df['off_workday'] = ~df['ds'].apply(is_workday)
  m.add_seasonality(name='weekly_on_workday', period=7, fourier_order=10, condition_name='on_workday')
  m.add_seasonality(name='weekly_off_workday', period=7, fourier_order=3, condition_name='off_workday')
  # print(df)

  m.fit(df)


  # Provisional
  # print('porvisional')
  future_aemet['on_workday'] = future_aemet['ds'].apply(is_workday)
  future_aemet['off_workday'] = ~future_aemet['ds'].apply(is_workday)
  # print(future_aemet)

  forecast = m.predict(future_aemet)
  print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']])

  if options and options["cross_validations"]:
    # df_cv = cross_validation(m, initial='730 days', period='180 days', horizon = '365 days', parallel="processes")
    df_cv = cross_validation(m, horizon = '365 days', parallel="processes")
    df_p = performance_metrics(df_cv)
    print(df_p)
    if options["figs_cross_validations"]:
      for fig in options["figs_cross_validations"]:
        plot_cross_validation_metric(df_cv, metric = fig)


  if options and options["plot_figures"]:
    fig1 = m.plot(forecast)
    fig2 = m.plot_components(forecast)
    # fig3 = plot_plotly(m, forecast)
  
  return forecast

In [ ]:
def predict_one_magnitude_with_aemet(file_name_magnitude, magnitude, file_name_aemet, columns_to_filter, periods_number, options):
  df_magnitude = prophet_data_normalized_one_magnitude(file_name_magnitude, magnitude)
  df_aemet = normalize_aemet_data(file_name_aemet, columns_to_filter)
  df_merged = pd.merge(df_magnitude, df_aemet, on='ds')

  future_aemet = filter_aemet_data_by_date(df_aemet, df_merged['ds'][0], '2020-01-07', columns_to_filter)

  table = filter_columns(df_merged, ['ds', 'y'])

  forecast = prophet_prediction_with_regresors(df_merged, periods_number, columns_to_filter, future_aemet, options)
  
  return {'forecast': forecast, 'df_merged': df_merged}